# SOLID Principles I

## Single Responsibility Principle (SRP)

The **Single Responsibility Principle** states that a class should have **only one reason to change**. This means a class should only have one job or responsibility. If a class handles multiple, unrelated tasks, a change in one task could inadvertently break the others.

By following SRP, your code becomes easier to understand, maintain, and test because each class is focused on a specific, well-defined purpose.

### Bad Example: Violating SRP

Consider a class that handles both creating a report and saving that report to a file. This class has two distinct responsibilities: content generation and data persistence.

```python
# BAD: This class has two responsibilities.
# 1. Generating report content.
# 2. Saving the report to a file.

class Report:
    def __init__(self, title, content):
        self.title = title
        self.content = content

    def generate_report(self):
        """Generates the report's string format."""
        return f"Report Title: {self.title}\n\n{self.content}"

    def save_to_file(self, filename):
        """Saves the report to a file."""
        # This part handles file I/O, which is a separate concern.
        # A change in file saving logic (e.g., changing format to JSON,
        # saving to a database) would force this class to change.
        report_str = self.generate_report()
        with open(filename, 'w') as f:
            f.write(report_str)

# Usage
my_report = Report("Quarterly Sales", "Sales are up by 15%!")
my_report.save_to_file("sales_report.txt")
```

The problem here is that the `Report` class knows about both report formatting and the file system. If we later decide to save reports to a database or send them over a network, we would have to modify this class, even though the report generation logic hasn't changed.

### Good Example: Following SRP

To fix this, we can split the responsibilities into two separate classes. One class will handle report generation, and the other will handle persistence.

```python
# GOOD: Each class has a single, well-defined responsibility.

class Report:
    """Handles creating the report content."""
    def __init__(self, title, content):
        self.title = title
        self.content = content

    def generate(self):
        """Generates the report's string format."""
        return f"Report Title: {self.title}\n\n{self.content}"

class FileManager:
    """Handles saving data to a file."""
    @staticmethod
    def save(data, filename):
        """Saves data to a specified file."""
        with open(filename, 'w') as f:
            f.write(data)

# Usage
# Now the responsibilities are cleanly separated.
sales_report = Report("Quarterly Sales", "Sales are up by 15%!")
report_content = sales_report.generate()

# The FileManager can be used to save any kind of data, not just reports.
FileManager.save(report_content, "sales_report.txt")
```


Now, the `Report` class is only responsible for creating report content. The `FileManager` is only responsible for saving data. If we need to change how files are saved, we only modify `FileManager`, leaving `Report` untouched.

## Open/Closed Principle (OCP)


The **Open/Closed Principle** states that software entities (classes, modules, functions) should be **open for extension, but closed for modification**. This means you should be able to add new functionality without changing existing, tested code.

This is usually achieved using abstractions, like interfaces or abstract base classes. You rely on these abstractions in your existing code and create new concrete classes to implement new features. This minimizes the risk of introducing bugs into the existing codebase.

### Bad Example: Violating OCP

Imagine a class that calculates discounts for different types of customers. A common but poor implementation uses a series of `if/elif` statements.

```python
# BAD: This class must be modified every time a new customer type is added.

class DiscountCalculator:
    def calculate(self, customer_type, price):
        """Calculates the discount based on customer type."""
        discount = 0
        if customer_type == 'regular':
            discount = price * 0.1  # 10% discount
        elif customer_type == 'vip':
            discount = price * 0.2  # 20% discount
        # What if we add a 'gold' customer type? We MUST modify this method!
        # This makes the class brittle and hard to maintain.
        return price - discount

# Usage
calculator = DiscountCalculator()
final_price_regular = calculator.calculate('regular', 100)
final_price_vip = calculator.calculate('vip', 100)

print(f"Regular Customer Price: ${final_price_regular}") # $90.0
print(f"VIP Customer Price: ${final_price_vip}")       # $80.0
```

The problem is that every time a new customer discount type is introduced (e.g., 'gold', 'platinum'), we have to go back and **modify** the `calculate` method by adding another `elif` block. This violates the "closed for modification" rule.

### Good Example: Following OCP

A better approach is to use a strategy pattern with an abstract base class. The main calculator will work with an abstraction, and we can add new discount types by creating new classes that implement this abstraction.

```python
# GOOD: Open for extension (new strategies), closed for modification (calculator).

from abc import ABC, abstractmethod

class DiscountStrategy(ABC):
    """Abstract base class for all discount strategies."""
    @abstractmethod
    def apply_discount(self, price):
        pass

# --- Concrete Implementations (Extensions) ---
class RegularCustomerDiscount(DiscountStrategy):
    """10% discount for regular customers."""
    def apply_discount(self, price):
        return price * 0.9

class VIPCustomerDiscount(DiscountStrategy):
    """20% discount for VIP customers."""
    def apply_discount(self, price):
        return price * 0.8

# We can easily add a new discount type without touching existing code.
class GoldCustomerDiscount(DiscountStrategy):
    """30% discount for Gold customers."""
    def apply_discount(self, price):
        return price * 0.7

# --- The Class Closed for Modification ---
class DiscountCalculator:
    def calculate(self, strategy: DiscountStrategy, price):
        """Calculates the final price using a given strategy."""
        return strategy.apply_discount(price)

# Usage
calculator = DiscountCalculator()
price = 100

# Calculate for different customer types by passing different strategy objects.
final_price_regular = calculator.calculate(RegularCustomerDiscount(), price)
final_price_vip = calculator.calculate(VIPCustomerDiscount(), price)
final_price_gold = calculator.calculate(GoldCustomerDiscount(), price) # Newly added!

print(f"Regular Customer Price: ${final_price_regular}") # $90.0
print(f"VIP Customer Price: ${final_price_vip}")       # $80.0
print(f"Gold Customer Price: ${final_price_gold}")       # $70.0
```

Here, the `DiscountCalculator` is **closed for modification**. Its `calculate` method will never need to change. However, the system is **open for extension**. We can introduce as many new discount strategies as we want simply by creating new classes that inherit from `DiscountStrategy`. This is a much more robust and scalable design.